Use beautiful soup package to scrape a wiki page and move the data to a dataframe.

In [1]:
#Importing packages
get_ipython().system(u' pip install --upgrade pip')
get_ipython().system(u' pip install beautifulsoup4')
!pip install lxml
!pip install html5lib
!pip install requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
print('Libraries imported.')

Requirement already up-to-date: pip in /opt/conda/envs/Python36/lib/python3.6/site-packages (19.3.1)
Libraries imported.


In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
table_canada = soup.find('table', {"class":"wikitable"})
#print(table_canada)
canada = []
for row in table_canada.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 3:
        canada.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))
canada_array = np.asarray(canada)
len(canada_array)


287

In [3]:
df = pd.DataFrame(canada_array)
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [4]:
groupby_df = pd.DataFrame({'Neighborhood' : df.groupby( ['PostalCode','Borough'])['Neighborhood'].apply(','.join)}).reset_index()

In [5]:
groupby_df.shape

(103, 3)

# Link 1 Task Finished!

Upload csv file provided since geocoder and geopy didn't work. Merge the longitude and latitude data with the wiki canada data. 

In [6]:
!pip install geocoder # import geocoder
!pip install geopy #geocoders with google didn't work so I had to use this as well

In [7]:
# @hidden_cell 
# The following code contains the credentials for a file in your IBM Cloud Object Storage. 
# I commented out my credentials so if it doen't work, that's why. 
credentials_1 = { 
    'IAM_SERVICE_ID': 'Insert', 
    'IBM_API_KEY_ID': 'Insert',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com', 
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token', 
    'BUCKET': 'ibmcourseraweek3capstonecanadadat-donotdelete-pr-1tub5f5umxdyu7', 
    'FILE': 'Geospatial_Coordinates (3).csv' }

In [8]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])

In [9]:
#cos.upload_file(Filename='wine/wine.csv',Bucket=credentials['BUCKET'],Key='wine_data.csv')


In [11]:
import types 
import pandas as pd 
from botocore.client import Config 
import ibm_boto3 
def __iter__(self): 
    return 0 
# @hidden_cell # The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials. 
# I commented out my credentials so if it doen't work, that's why. 
client_insertclientname = ibm_boto3.client(service_name='s3', 
ibm_api_key_id='Insert', 
ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
config=Config(signature_version='oauth'),
endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com') 
body = client_d60d72f57ebd49a4b3e31b41536ee6a4.get_object(Bucket='ibmcourseraweek3capstonecanadadat-donotdelete-pr-1tub5f5umxdyu7',Key='Geospatial_Coordinates (3).csv')['Body'] 
# add missing __iter__ method, so pandas accepts body as file-like object if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body ) 
# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement. df_data_0 = pd.read_csv(body) df_data_0.head() 

In [12]:
latlong = pd.read_csv(body)

In [13]:
latlong.columns = ['PostalCode','Latitude','Longitude']
latlong.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [14]:
#groupby_df
#latlong
Canada_DF = pd.merge(groupby_df,latlong, left_on='PostalCode',right_on='PostalCode',how='left')
display(Canada_DF.head(10))

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


# Link 2 Task Finished!

The next section is making a map of the previously created data.

In [15]:
#import numpy as np # library to handle data in a vectorized manner
#import pandas as pd # library for data analsysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('More Libraries imported.')

Solving environment: done

# All requested packages already installed.

More Libraries imported.


In [18]:
address = 'Canada'
geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Canada are 61.0666922, -107.9917071.


Make dataframe into a map

In [19]:
import folium # map rendering library
# create map of Canada using latitude and longitude values
#latlong.columns = ['PostalCode','Latitude','Longitude']
map_Canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for Latitude, Longitude, Borough, Neighborhood in zip(Canada_DF['Latitude'], Canada_DF['Longitude'], Canada_DF['Borough'], Canada_DF['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada)
display(map_Canada)
#map_Canada.display()

# Link 3 Task Finished!

The sample code in the NY example no longer displays a map so when I use it here I have the same issue... Folium code in NY example needs correcting.